In [2]:
import os
import sys
import platform
import re
import json
import subprocess

In [4]:
def is_empty(variable):
    if variable is None:
        return True
    if isinstance(variable, str) and variable == "":
        return True
    if isinstance(variable, (list, tuple, dict, set)) and len(variable) == 0:
        return True
    return False

In [5]:
# subprocess.call()：执行命令，并返回执行状态，其中shell参数为False时，命令以及命令的参数需要通过列表的方式传入，当shell为True时，可通过一个字符串直接传入命令以及命令所需要的参数
# print(subprocess.call(["touch","c.txt"],shell=False))  # 直接返回0，表示操作成功
# print(subprocess.call("mkdir newdir1",shell=True))   # 直接返回0，表示操作成功
print(subprocess.call(["powershell","-noprofile","-c","ls"],shell=False))  # shell参数为false，则，命令以及参数以列表的形式给出


0


In [7]:
print(subprocess.call(["powershell -noprofile -c ls"],shell=True))

1


In [11]:
a=subprocess.check_output(["powershell","-noprofile","-c","ls"],shell=False)
print(a)  # 不是直接返回0了，而是直接返回执行结果的内容


b'\r\n\r\n    \xc4\xbf\xc2\xbc: E:\\code\\project\\config\\shells-setup\\test\r\n\r\n\r\nMode                 LastWriteTime         Length Name                                                                 \r\n----                 -------------         ------ ----                                                                 \r\n-a----        10/27/2023   9:28 PM          13608 test.ipynb                                                           \r\n-a----        10/25/2023   9:26 PM           1510 test.ps1                                                             \r\n-a----        10/27/2023   9:01 PM              0 test.py                                                              \r\n\r\n\r\n'


In [12]:
b=subprocess.check_output("powershell -noprofile -c ls",shell=True)
print(b)  # 不是直接返回0了，而是直接返回执行结果的内容

b'\r\n\r\n    \xc4\xbf\xc2\xbc: E:\\code\\project\\config\\shells-setup\\test\r\n\r\n\r\nMode                 LastWriteTime         Length Name                                                                 \r\n----                 -------------         ------ ----                                                                 \r\n-a----        10/27/2023   9:28 PM          13608 test.ipynb                                                           \r\n-a----        10/25/2023   9:26 PM           1510 test.ps1                                                             \r\n-a----        10/27/2023   9:01 PM              0 test.py                                                              \r\n\r\n\r\n'


In [4]:

f=os.popen("dir")  # 返回的是一个文件对象
print(f.read())    

 驱动器 E 中的卷是 Documents
 卷的序列号是 A9CC-C676

 e:\code\project\config\shells-setup\test 的目录

10/27/2023  09:01 PM    <DIR>          .
10/27/2023  09:00 PM    <DIR>          ..
10/27/2023  09:21 PM             7,501 test.ipynb
10/25/2023  09:26 PM             1,510 test.ps1
10/27/2023  09:01 PM                 0 test.py
               3 个文件          9,011 字节
               2 个目录 194,886,070,272 可用字节



In [2]:
import subprocess
def run_shell_command(command: str = "", shell: str = "powershell") -> int:
    if shell == "powershell":
        command = "powershell.exe -noprofile -c " + command
    elif shell == "pwsh":
        command = "pwsh.exe -noprofile -c " + command
    command_array = command.split(" ")
    # https://blog.51cto.com/u_16175493/7917058
    # result = subprocess.run(command_array, capture_output=True, text=True)
    result = subprocess.run(command_array, capture_output=True, shell=True, text=True)
    if result.returncode == 0:
        print('命令执行成功！')
        print('命令的输出结果为：')
        print(result.stdout)
    else:
        print('命令执行失败！')
        print('命令的错误输出为：')
        print(result.stderr)
    return result.returncode

In [3]:

run_shell_command("ls")
# run_shell_command("mkdir a,b")
# run_shell_command("gcm choco", "pwsh")
# run_shell_command("dir", "cmd")

命令执行成功！
命令的输出结果为：


    目录: E:\code\project\config\shells-setup\test


Mode                 LastWriteTime         Length Name                                                                 
----                 -------------         ------ ----                                                                 
-a----        10/27/2023   9:02 PM           5491 test.ipynb                                                           
-a----        10/25/2023   9:26 PM           1510 test.ps1                                                             
-a----        10/27/2023   9:01 PM              0 test.py                                                              





0

In [1]:
import subprocess
def run_shell_script(path: str = "", shell: str = "powershell") -> object:
    try:
        args=["powershell.exe", "-noprofile", "-c", path]
        p=subprocess.Popen(args, stdout=subprocess.PIPE)
        dt=p.stdout.read().decode()
        return dt
    except Exception as e:
        print(e)
    return False

In [2]:
print(run_shell_script(r"./test.ps1"))

In [ ]:
import subprocess

def python_call_powershell():
    try:
        args=["powershell.exe","-noprofile","-c",r"./test.ps1"]  #args参数里的ip是对应调用powershell里的动态参数args[0],类似python中的sys.argv[1]
        p=subprocess.Popen(args, stdout=subprocess.PIPE)
        dt=p.stdout.read().decode()
        return dt
    except Exception as e:
        print(e)
    return False
if __name__=="__main__":
    print(type(python_call_powershell()))
    print(python_call_powershell())

In [1]:
def run_shell_script_with_admin(path: str = "", shell: str = "powershell") -> int:
    p = subprocess.Popen(
            [
                'powershell.exe', 
                "-noprofile", "-c",
                r"""
                exit (
                Start-Process -Verb RunAs -PassThru -Wait powershell.exe -Args "
                    -noprofile -c Set-Location \`"$PWD\`"; & .\test.ps1; exit `$LASTEXITCODE
                "
                ).ExitCode
                """
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
    p.communicate()
    print('Terminated with exit code ' + str(p.returncode))
    return p.returncode

In [ ]:
pscommand = 'Invoke-Command ...'
process = subprocess.Popen(['powershell.exe', '-NoProfile', '-Command', '"&{' + pscommand + '}"'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [14]:
import os
from glob import glob
import subprocess as sp


class PowerShell:
    # from scapy
    def __init__(self, coding, ):
        cmd = [self._where('PowerShell.exe'),
               "-NoLogo", "-NonInteractive",  # Do not print headers
               "-Command", "-"]  # Listen commands from stdin
        startupinfo = sp.STARTUPINFO()
        startupinfo.dwFlags |= sp.STARTF_USESHOWWINDOW
        self.popen = sp.Popen(cmd, stdout=sp.PIPE, stdin=sp.PIPE, stderr=sp.STDOUT, startupinfo=startupinfo)
        self.coding = coding

    def __enter__(self):
        return self

    def __exit__(self, a, b, c):
        self.popen.kill()

    def run(self, cmd, timeout=15):
        b_cmd = cmd.encode(encoding=self.coding)
        try:
            b_outs, errs = self.popen.communicate(b_cmd, timeout=timeout)
        except sp.TimeoutExpired:
            self.popen.kill()
            b_outs, errs = self.popen.communicate()
        outs = b_outs.decode(encoding=self.coding)
        return outs, errs

    @staticmethod
    def _where(filename, dirs=None, env="PATH"):
        """Find file in current dir, in deep_lookup cache or in system path"""
        if dirs is None:
            dirs = []
        if not isinstance(dirs, list):
            dirs = [dirs]
        if glob(filename):
            return filename
        paths = [os.curdir] + os.environ[env].split(os.path.pathsep) + dirs
        try:
            return next(os.path.normpath(match)
                        for path in paths
                        for match in glob(os.path.join(path, filename))
                        if match)
        except (StopIteration, RuntimeError):
            raise IOError("File not found: %s" % filename)


if __name__ == '__main__':
    # Example:
    with PowerShell('GBK') as ps:
        outs, errs = ps.run('ping baidu.com')
    print('error:', os.linesep, errs)
    print('output:', os.linesep, outs)

error: 
 None
output: 
 Set-PSReadLineOption : 句柄无效。
所在位置 C:\Users\lgf\Documents\WindowsPowerShell\Microsoft.PowerShell_profile.ps1:22 字符: 1
+ Set-PSReadLineOption -PredictionViewstyle Listview
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : NotSpecified: (:) [Set-PSReadLineOption], IOException
    + FullyQualifiedErrorId : System.IO.IOException,Microsoft.PowerShell.SetPSReadLineOption
 
Set-PSReadLineOption : The predictive suggestion feature cannot be enabled because the console output doesn't support 
virtual terminal processing or it's redirected.
所在位置 C:\Users\lgf\Documents\WindowsPowerShell\Microsoft.PowerShell_profile.ps1:24 字符: 1
+ Set-PSReadLineOption -PredictionSource History
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : NotSpecified: (:) [Set-PSReadLineOption], ArgumentException
    + FullyQualifiedErrorId : System.ArgumentException,Microsoft.PowerShell.SetPSReadLineOption
 

正在 Ping baidu.com [39.156.66.10]

In [ ]:
import ctypes, sys
class CMD(object):
    """
    运行windows shell 命令
    """
    __instance__ = None

    def __new__(cls, *args, **kwargs):
        """
        实现单例模式
        """
        if cls.__instance__ is None:
            cls.__instance__ = super().__new__(cls, *args, **kwargs)
        return cls.__instance__

    @staticmethod
    def is_admin():
        """
        判断是否admin
        """
        try:
            return ctypes.windll.shell32.IsUserAnAdmin()
        except Exception as e:
            logger.error("Error: ", e)
            return False

    def run_cmd(self, cmd):
        """
        windows下运行shell命令
        """
        cmd_list = [cmd] if isinstance(cmd, str) else cmd

        if self.is_admin:
            # 此处运行需要运行的命令
            for c in cmd_list:
                result_status, result_str = subprocess.getstatusoutput(c)
                logger.info(f"CMD>>>>> {c}, status: {result_status}, {result_str}")
                return result_status, result_str
        else:
            ctypes.windll.shell32.ShellExecuteW(None, "runas", sys.executable, __file__, None, 1)

In [1]:
# -*- coding: utf-8 -*-
import subprocess

def python_call_powershell(ip):
    try:
        args=[r"powershell",r".\test.ps1",ip]  #args参数里的ip是对应调用powershell里的动态参数args[0],类似python中的sys.argv[1]
        p=subprocess.Popen(args, stdout=subprocess.PIPE)
        dt=p.stdout.read()
        return dt
    except Exception as e:
        print(e)
    return False

if __name__=="__main__":
    ip=["1.1.1.1","2.2.2.2","3.3.3.3"]
    print(python_call_powershell(ip))

expected str, bytes or os.PathLike object, not list
False


In [2]:
def python_call_powershell(ip):
    try:
        args=[r"powershell.exe","-ExecutionPolicy","Unrestricted", r".\test.ps1",ip]
        p=subprocess.Popen(args, stdout=subprocess.PIPE)
        dt=p.stdout.read()
        return dt
    except Exception as e:
        print(e)
    return False

if __name__=="__main__":
    ip=["1.1.1.1","2.2.2.2","3.3.3.3"]
    print(python_call_powershell(ip))
    
    

expected str, bytes or os.PathLike object, not list
False


In [ ]:
# -*- coding: utf-8 -*-
import wmi,json
import time

logfile = 'logs_%s.txt' % time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime())

#远程执行bat文件
def call_remote_bat(ipaddress,username,password):
    try:
        #用wmi连接到远程服务器
        conn = wmi.WMI(computer=ipaddress, user=username, password=password)
        filename=r"D:\apps\autorun.bat"   #此文件在远程服务器上
        cmd_callbat=r"cmd /c call %s"%filename
        conn.Win32_Process.Create(CommandLine=cmd_callbat)  #执行bat文件
        print("执行成功!")
        return True
    except Exception,e:
        log = open(logfile, 'a')
        log.write(('%s, call bat Failed!\r\n') % ipaddress)
        log.close()
        return False
    return False

if __name__=='__main__':
    call_remote_bat(computer="192.168.1.2", user="testuser", password="testpwd")

In [24]:
# -*- coding: utf-8 -*-
import subprocess

def python_call_powershell():
    try:
        args=["powershell.exe","-noprofile","-c",r"./test.ps1"]  #args参数里的ip是对应调用powershell里的动态参数args[0],类似python中的sys.argv[1]
        p=subprocess.Popen(args, stdout=subprocess.PIPE)
        dt=p.stdout.read().decode()
        return dt
    except Exception as e:
        print(e)
    return False
if __name__=="__main__":
    print(type(python_call_powershell()))
    print(python_call_powershell())

# 如果报错，说禁止执行脚本，是因为没有权限，所以，把上面的一行代码换成
#  args=[r"C:\WINDOWS\system32\WindowsPowerShell\v1.0\powershell.exe","-ExecutionPolicy","Unrestricted", r"D:\jzhou\test_ping.ps1",ip]

<class 'str'>
hello, lgf



In [27]:
# -*- coding: utf-8 -*-
import subprocess

def python_call_powershell():
    try:
        args=["powershell.exe","-noprofile","-c",
              r"""
                exit (
                Start-Process -Verb RunAs -PassThru -Wait powershell.exe -Args "
                    -noprofile -c Set-Location \`"$PWD\`"; & .\test.ps1; exit `$LASTEXITCODE
                "
                ).ExitCode
                """
              ]
        p=subprocess.Popen(args, stdout=subprocess.PIPE)
        dt=p.stdout.read().decode()
        return dt
    except Exception as e:
        print(e)
    return False
if __name__=="__main__":
    print(type(python_call_powershell()))
    print(python_call_powershell())

<class 'str'>



In [32]:
p = subprocess.Popen(
  [
    "powershell.exe", 
    "-noprofile", "-c",
    r"""
    exit (
      Start-Process -Verb RunAs -PassThru -Wait powershell.exe -Args "
        -noprofile -c Set-Location \`"$PWD\`"; & .\test.ps1; exit `$LASTEXITCODE
      "
    ).ExitCode
    """
  ],
  stdout=subprocess.PIPE
)
p.communicate()

print('Terminated with exit code ' + str(p.returncode))

Terminated with exit code 3221225786


In [33]:
import subprocess, sys
p = subprocess.Popen(["powershell.exe", r".\test.ps1"], stdout=subprocess.PIPE)
p.communicate()

In [18]:
import subprocess, sys
p = subprocess.Popen('powershell.exe -ExecutionPolicy RemoteSigned -file "E:\code\project\config\shells-setup\test\test.ps1"', stdout=sys.stdout)
p.communicate()

UnsupportedOperation: fileno